In [ ]:
import os
import numpy as np
import pickle
import shutil 
import pImpactR as impact
import matplotlib.pyplot as plt
import time

# Simulation Setting

In [ ]:
npt = 1024
Espread = 5.0e-3
Rsphere = 0.45
pipe_radius = 0.02

# Lattice

In [ ]:
beam,lattice=impact.readInputFile('IOTA.Chris.in')
beam.nCore_y = 1
beam.nCore_z = 1

In [ ]:
lattice = [ item for item in lattice if not item.type == 'write_raw_ptcl' ]
lattice = [ item for item in lattice if not item.type == '-8' ]
lattice[0].turns = 1
for i in range(len(lattice)):
    if 'length' in lattice[i].keys():
        lattice[i].n_sckick = int(np.ceil(lattice[i].length*40))
        lattice[i].n_map = 1
    if 'pipe_radius' in lattice[i].keys() and lattice[i].type!='dipole':
        lattice[i].pipe_radius = pipe_radius

elemWrite = impact.getElem('write_raw_ptcl')
elemWrite.file_id   = 999
elemWrite.format_id = 2
lattice.append(elemWrite)

In [ ]:
NL_nu = lattice[1].tune_advance
NL_L  = lattice[1].length
NL_c  = 0.01
alfx = np.tan(np.pi*NL_nu)
betx = NL_L/np.sin(2.0*np.pi*NL_nu)
k = 2*alfx/betx
k

In [ ]:
arc = lattice.copy()
loop = arc[0]
nlinsert = arc.pop(1)

In [ ]:
nlinsert.n_sckick = 2
nlinsert.n_map = 50

In [ ]:
ke = beam.kinetic_energy
freq = beam.frequency
mass = beam.mass
gam0 = 1.0+beam.kinetic_energy/mass
bet0 = np.sqrt(1.0-1.0/gam0**2)
bg0  = np.sqrt(gam0**2-1.0)
q_m  = beam.multi_charge.q_m[0]

In [ ]:
def Impact2norm(data_in,bg0,bet0,sign=1):
    data=data_in.copy()
    data[:,5] = -(np.sqrt(1.0-2.0*data[:,5]/mass/(bet0*bg0)+(data[:,5]/mass)**2/bg0**2)-1.0)
    data[:,1] = (data[:,0]*alfx*sign/np.sqrt(betx) + data[:,1]/(1+data[:,5])*np.sqrt(betx))/NL_c
    data[:,3] = (data[:,2]*alfx*sign/np.sqrt(betx) + data[:,3]/(1+data[:,5])*np.sqrt(betx))/NL_c
    data[:,0] = data[:,0]/(np.sqrt(betx)*NL_c)
    data[:,2] = data[:,2]/(np.sqrt(betx)*NL_c)
    return data
    
def norm2Impact(data_in,bg0,bet0,sign=1):
    data=data_in.copy()
    data[:,1] = (-data[:,0]*alfx*sign + data[:,1])*NL_c/np.sqrt(betx)*(1+data[:,5])
    data[:,3] = (-data[:,2]*alfx*sign + data[:,3])*NL_c/np.sqrt(betx)*(1+data[:,5])
    data[:,0] = data[:,0]*np.sqrt(betx)*NL_c
    data[:,2] = data[:,2]*np.sqrt(betx)*NL_c
    data[:,5] = (bg0*np.sqrt(1/bet0**2+2.0*data[:,5]+data[:,5]**2)-bg0/bet0)*mass
    return data

In [ ]:
SextIndex=[]
SextStrength=[]
for i in range(len(arc)):
    if arc[i]['type']=='multipole_thin':
        SextIndex.append(i)
        SextStrength.append(arc[i]['KL_sext'])
nParam = len(SextIndex)

In [ ]:
#%%
pData=np.zeros([npt,9])
pData[:,6]  = q_m
pData[:,-1] = np.arange(1,npt+1)
pData[0,0]  = 0.45
pData[1,0]  = -0.45
normFactor = 1.0/(4*npt*Rsphere*Rsphere)

# refp = pData[:4,:]
# refp[0,0] = 0.45
# refp[1,0] = -0.45
# refpImpact = norm2Impact(refp,bg0,bet0,1)

def getLost():
    with open('lost_partcl.data') as f:
        for i, l in enumerate(f):
            pass
    return i

def objFunc(arg): 
    obj = 0.001*np.sum(np.array(arg)*np.array(arg))/nParam
    if obj > 0.001:
        return 1.0
    
    target = impact.opt.id_generator()  # generage random directory name
    while os.path.exists(target):  
        target = impact.opt.id_generator()
    shutil.copytree('origin', target)
    os.chdir(target) # cd to the randome directory and
    
    for i,j in enumerate(SextIndex):
        arc[j]['KL_sext']=200.0*arg[i]
#         arc[j]['KL_sext']=1.0e3*arg[2*i]
#         arc[j]['KL_oct'] =1.0e6*arg[2*i+1]
        
    x = np.random.random([npt-2,4])
    c0 = np.cos(x[:,0])
    s0 = np.sin(x[:,0])
    c1 = np.cos(x[:,1])
    s1 = np.sin(x[:,1])
    c2 = np.cos(x[:,2])
    s2 = np.sin(x[:,2])
    pData[2:,0] = Rsphere*c0
    pData[2:,1] = Rsphere*s0*c1
    pData[2:,2] = Rsphere*s0*s1*c2
    pData[2:,3] = Rsphere*s0*s1*s2
    pData[2:,5] = (x[:,3]-0.5)*Espread

    beam.n_particles = npt
    loop.turns = 1
    impact.writeParticleData(norm2Impact(pData,bg0,bet0,-1),ke,mass,freq)
    impact.writeInputFile(beam,arc)
    impact.run()
    nLost = getLost()
    if 0 < nLost:
        os.chdir('..')
        shutil.rmtree(target)
        return 1.0*nLost
    pData2 = impact.readParticleData(999, ke, mass, freq, format_id=2)
    pData2 = Impact2norm(pData2,bg0,bet0,1)
    obj = obj + np.sqrt(np.sum((pData[:,:4]-pData2[:,:4])**2)*normFactor)
#     !mv log.impact_std log.impact_std.0
    
#     beam.n_particles = 4
#     loop.turns = 10
#     impact.writeParticleData(norm2Impact(pData,bg0,bet0,1),ke,mass,freq)
# #     impact.writeParticleData(refpImpact,ke,mass,freq)
#     lattice = arc[:-1].copy()
#     lattice.insert(1,nlinsert)
#     impact.writeInputFile(beam,lattice)
#     impact.run()
#     if 0 < getLost():
#         os.chdir('..')
#         shutil.rmtree(target)
#         return 1.0e22
#     !mv log.impact_std log.impact_std.1
    
    os.chdir('..')
    shutil.rmtree(target)
    return obj

In [ ]:
objFunc([0]*nParam)

In [ ]:
#%% run optim
bounds = [(-1.0,1.0)]*nParam
# result=impact.opt.differential_evolution(objFunc, bounds, ncore=32, popsize=64, 
#                                         disp=True, polish=False, maxtime=60*20) 
#                                         # stop running at maximum 1 min
# print(result)
# with open('result.sphere.12sext','wb') as fp:
#     pickle.dump(result,fp)

In [ ]:
with open('result.sphere.12sext','rb') as f:
    result = pickle.load(f)

In [ ]:
while True:
    previous_result = result
    result = impact.opt.differential_evolution(objFunc, bounds, ncore=32, 
                                           prev_result=previous_result, 
                                           disp=True, polish=False, maxtime=60*20)
    with open('result.sphere.12sext','wb') as fp:
        pickle.dump(result,fp)
        if hasattr(result,'x'): 
            break